# **Haiku Generator**
## Cem Ayerdem and Tolunay Ahmedov

User input is used to generate a unique Haiku with the users theme/words in mind


In [51]:
!pip install pronouncing
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import re
import random

# Function to generate Haiku's
def generate_haiku(model, seed, max_length=max_seq_length):
    seed = seed.ljust(max_length)[:max_length]

    result, pattern = [], [char_to_int[char] if char in char_to_int else char_to_int[' '] for char in seed.lower()]

    while len(result) < max_length:
        x = np.reshape(pattern, (1, len(pattern), 1))
        x = x / float(n_vocab)
        prediction = model.predict(x, verbose=0)
        index = np.argmax(prediction)
        result.append(int_to_char[index])
        pattern.append(index)
        pattern = pattern[1:]

    return ''.join(result)

# Function to extract unique words from the dataset (inspiration)
def get_dataset_words(processed_haikus, num_words=100):
    words = set()
    for haiku in processed_haikus:
        words.update(haiku.split())
    return list(words)[:num_words]

processed_haikus = [" ".join(re.sub(r'[^a-z\s]', '', haiku).split()) for haiku in haikus]

# Extract words from the processed Haiku's
unique_words = get_dataset_words(processed_haikus, num_words=100)

# Print the unique words from random Haiku's
print("Unique words in the dataset:")
print(unique_words)


# 
# Create a seed sequence from user and dataset words
def create_seed_sequence(user_words, dataset_words, num_user_words=2, num_dataset_words=2):
    selected_user_words = random.sample(user_words, min(num_user_words, len(user_words)))
    selected_dataset_words = random.sample(dataset_words, num_dataset_words)

    seed_words = selected_user_words + selected_dataset_words
    random.shuffle(seed_words)

    return ' '.join(seed_words)

# Read and split the Haiku's
with open("haiku.txt", "r") as file:
    haikus = file.read().lower().split("\n\n")

# Process Haiku
processed_haikus = [" ".join(re.sub(r'[^a-z\s]', '', haiku).split()) for haiku in haikus]

# Determine fixed sequence length
max_seq_length = 40

# Create unique character map
chars = sorted(list(set(" ".join(processed_haikus))))
char_to_int = {char: i for i, char in enumerate(chars)}
int_to_char = {i: char for i, char in enumerate(chars)}

# Create training data
X_data, y_data = [], []
for haiku in processed_haikus:
    for i in range(0, len(haiku) - max_seq_length, 1):
        X = haiku[i:i + max_seq_length]
        y = haiku[i + max_seq_length] if i + max_seq_length < len(haiku) else ' '
        X_data.append([char_to_int[char] for char in X])
        y_data.append(char_to_int[y])

n_patterns = len(X_data)
n_vocab = len(chars)

# Reshape and normalize the input data
X = np.reshape(X_data, (n_patterns, max_seq_length, 1))
X = X / float(n_vocab)

y = to_categorical(y_data, num_classes=n_vocab)

# Define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(max_seq_length, 1), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(n_vocab, activation='softmax'))
optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

# Train model
model.fit(X, y, epochs=20, batch_size=64)

# Function to  modify a Haiku with user-input words
def modify_haiku_with_user_words(haikus, user_words, num_words_to_add=2):
    # Randomly select a Haiku
    selected_haiku = random.choice(haikus)
    words = selected_haiku.split()

    # Insert user words
    for _ in range(min(num_words_to_add, len(user_words))):
        insert_pos = random.randint(0, len(words))
        words.insert(insert_pos, random.choice(user_words))

    return ' '.join(words)

# Read and split
with open("haiku.txt", "r") as file:
    haikus = file.read().lower().split("\n\n")

# Process Haiku's
processed_haikus = [" ".join(re.sub(r'[^a-z\s]', '', haiku).split()) for haiku in haikus]

# User input for words
user_input = input("Hi Writer! Welcome to the flexible Haiku generator made by Cem Ayerdem and Tolunay Ahmedov! \n Enter words to include in the Haikus (comma-separated): ")
user_words = user_input.replace(',', ' ').split()

# Build modded Haiku
modified_haiku = modify_haiku_with_user_words(processed_haikus, user_words)

print("Your unique Haiku:")
print(modified_haiku)


Unique words in the dataset:
['cooking', 'pension', 'rainbows', 'plan', 'only', 'north', 'streetlamps', 'that', 'solar', 'i', 'am', 'sealed', 'dusk', 'he', 'ageing', 'off', 'chemo', 'point', 'cleaning', 'sparrows', 'gray', 'betelgeuse', 'lifting', 'sunlight', 'rise', 'skin', 'day', 'place', 'prairie', 'sudden', 'her', 'water', 'still', 'willows', 'meet', 'thin', 'rain', 'hole', 'burning', 'gold', 'guest', 'sipping', 'pause', 'roses', 'dont', 'bay', 'strategic', 'side', 'recall', 'boats', 'call', 'just', 'small', 'warmth', 'long', 'homeland', 'anchor', 'trails', 'bag', 'distance', 'early', 'sand', 'beside', 'local', 'frames', 'market', 'spring', 'among', 'window', 'high', 'strides', 'land', 'then', 'eases', 'winter', 'room', 'trail', 'flare', 'both', 'feathering', 'veiled', 'smile', 'my', 'halfhidden', 'head', 'the', 'top', 'february', 'warm', 'we', 'before', 'turning', 'out', 'warmedover', 'light', 'or', 'sends', 'left', 'two', 'fog']
Epoch 1/20
15/15 [==============================] -